In [1]:
import sys
sys.path.insert(0, "../..")

In [2]:
import torch
from torch import nn
from torch.utils import data
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
import numpy as np
from copy import deepcopy

from nn_extrapolation import AcceleratedSGD

In [3]:
torch.cuda.is_available()

True

In [4]:
val_loss_fn = nn.NLLLoss(reduction="sum")

def validation(model, loader):
    ok = 0
    loss_sum = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for x, y in tqdm(loader):
            x = x.cuda()
            y = y.cuda()
            out = model(x)
            loss_sum += val_loss_fn(out, y)
            preds = out.argmax(1)
            ok += (y == preds).sum()
            total += len(y)
    return ok / total, loss_sum / total

def train_epoch():
    model.train()
    param_hist = []
    for i, (x, y) in enumerate(tqdm(train_loader)):
        x = x.cuda()
        y = y.cuda()
        optimizer.zero_grad()
        out = model(x)
        loss = loss_fn(out, y)
        loss.backward()
        optimizer.step()
    return torch.tensor(np.mean(param_hist, 0))

In [5]:
train_ds = datasets.MNIST("../../../MNIST", download=True, train=True, transform=transforms.ToTensor())
test_ds = datasets.MNIST("../../../MNIST", download=True, train=False, transform=transforms.ToTensor())
valid_size = int(0.2 * len(train_ds))
train_ds, valid_ds = data.random_split(train_ds, [len(train_ds) - valid_size, valid_size])

train_loader = data.DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=2)
valid_loader = data.DataLoader(valid_ds, batch_size=64, shuffle=True, num_workers=2)
test_loader = data.DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=2)

In [6]:
def make_model():
    return nn.Sequential(
        nn.Conv2d(1, 32, 3),
        nn.ReLU(),
        nn.Conv2d(32, 32, 3),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Conv2d(32, 64, 3),
        nn.ReLU(),
        nn.Conv2d(64, 64, 3),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Flatten(),
        nn.Linear(4*4*64, 128),
        nn.ReLU(),
        nn.Linear(128, 10),
        nn.LogSoftmax(-1),
    )

In [7]:
model = make_model()
model.cuda()

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (6): ReLU()
  (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (8): ReLU()
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=1024, out_features=128, bias=True)
  (12): ReLU()
  (13): Linear(in_features=128, out_features=10, bias=True)
  (14): LogSoftmax(dim=-1)
)

In [8]:
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, momentum=0.5, weight_decay=1e-5, mode="epoch")
loss_fn = nn.NLLLoss()

## Epoch

In [9]:
epochs = 20

for epoch in range(epochs):
    print("Epoch", epoch+1)
    train_epoch()
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

Epoch 1


/opt/miniconda3/envs/torch/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/miniconda3/envs/torch/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



Validation accuracy: 0.1014, validation loss: 2.3010
Epoch 2



Validation accuracy: 0.1014, validation loss: 2.2973
Epoch 3



Validation accuracy: 0.1130, validation loss: 2.2909
Epoch 4


...


Validation accuracy: 0.9615, validation loss: 0.1182
Epoch 17



Validation accuracy: 0.9668, validation loss: 0.1085
Epoch 18



Validation accuracy: 0.9691, validation loss: 0.0995
Epoch 19



Validation accuracy: 0.9685, validation loss: 0.0969
Epoch 20



Validation accuracy: 0.9716, validation loss: 0.0901


In [10]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9745, device='cuda:0'), tensor(0.0835, device='cuda:0'))



Valid: (tensor(0.9716, device='cuda:0'), tensor(0.0901, device='cuda:0'))


In [11]:
optimizer.accelerate()

In [12]:
optimizer.store_parameters()
model.cuda()

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (6): ReLU()
  (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (8): ReLU()
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=1024, out_features=128, bias=True)
  (12): ReLU()
  (13): Linear(in_features=128, out_features=10, bias=True)
  (14): LogSoftmax(dim=-1)
)

In [13]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9756, device='cuda:0'), tensor(0.0792, device='cuda:0'))



Valid: (tensor(0.9726, device='cuda:0'), tensor(0.0860, device='cuda:0'))


## Epoch average

In [22]:
model = make_model()
model.cuda()
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, momentum=0.5, weight_decay=1e-5, mode="epoch_avg")

In [23]:
epochs = 20

for epoch in range(epochs):
    print("Epoch", epoch+1)
    train_epoch()
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

Epoch 1



Validation accuracy: 0.1494, validation loss: 2.3017
Epoch 2



Validation accuracy: 0.1402, validation loss: 2.2982
Epoch 3



Validation accuracy: 0.3442, validation loss: 2.2926
Epoch 4



Validation accuracy: 0.3957, validation loss: 2.2773
Epoch 5


...


Validation accuracy: 0.9638, validation loss: 0.1127
Epoch 17



Validation accuracy: 0.9670, validation loss: 0.1045
Epoch 18



Validation accuracy: 0.9698, validation loss: 0.0944
Epoch 19



Validation accuracy: 0.9667, validation loss: 0.0988
Epoch 20



Validation accuracy: 0.9721, validation loss: 0.0859


In [24]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9749, device='cuda:0'), tensor(0.0820, device='cuda:0'))



Valid: (tensor(0.9721, device='cuda:0'), tensor(0.0859, device='cuda:0'))


In [25]:
optimizer.accelerate()

In [26]:
optimizer.store_parameters()
model.cuda()

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (6): ReLU()
  (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (8): ReLU()
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=1024, out_features=128, bias=True)
  (12): ReLU()
  (13): Linear(in_features=128, out_features=10, bias=True)
  (14): LogSoftmax(dim=-1)
)

In [27]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9620, device='cuda:0'), tensor(0.1768, device='cuda:0'))



Valid: (tensor(0.9599, device='cuda:0'), tensor(0.1752, device='cuda:0'))


## Split + epoch

In [28]:
model = make_model()
model.cuda()
groups = [{"params": [param]} for param in model.parameters()]
optimizer = AcceleratedSGD(groups, 1e-3, k=10, momentum=0.5, weight_decay=1e-5, mode="epoch")

In [29]:
epochs = 20

for epoch in range(epochs):
    print("Epoch", epoch+1)
    train_epoch()
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

Epoch 1



Validation accuracy: 0.0896, validation loss: 2.3013
Epoch 2



Validation accuracy: 0.1954, validation loss: 2.2964
Epoch 3



Validation accuracy: 0.2359, validation loss: 2.2870
Epoch 4



Validation accuracy: 0.4085, validation loss: 2.2415
Epoch 5


...


Validation accuracy: 0.9667, validation loss: 0.1074
Epoch 17



Validation accuracy: 0.9694, validation loss: 0.0950
Epoch 18



Validation accuracy: 0.9707, validation loss: 0.0945
Epoch 19



Validation accuracy: 0.9732, validation loss: 0.0875
Epoch 20



Validation accuracy: 0.9751, validation loss: 0.0815


In [30]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9765, device='cuda:0'), tensor(0.0772, device='cuda:0'))



Valid: (tensor(0.9751, device='cuda:0'), tensor(0.0815, device='cuda:0'))


In [31]:
optimizer.accelerate()

In [32]:
optimizer.store_parameters()
model.cuda()

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (6): ReLU()
  (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (8): ReLU()
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=1024, out_features=128, bias=True)
  (12): ReLU()
  (13): Linear(in_features=128, out_features=10, bias=True)
  (14): LogSoftmax(dim=-1)
)

In [33]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9735, device='cuda:0'), tensor(0.0851, device='cuda:0'))



Valid: (tensor(0.9712, device='cuda:0'), tensor(0.0926, device='cuda:0'))


## Split + epoch average

In [34]:
model = make_model()
model.cuda()
groups = [{"params": [param]} for param in model.parameters()]
optimizer = AcceleratedSGD(groups, 1e-3, k=10, momentum=0.5, weight_decay=1e-5, mode="epoch")

In [35]:
epochs = 20

for epoch in range(epochs):
    print("Epoch", epoch+1)
    train_epoch()
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

Epoch 1



Validation accuracy: 0.1014, validation loss: 2.2984
Epoch 2



Validation accuracy: 0.1351, validation loss: 2.2883
Epoch 3



Validation accuracy: 0.4614, validation loss: 2.2466
Epoch 4



Validation accuracy: 0.7642, validation loss: 0.8764
Epoch 5


...


Validation accuracy: 0.9672, validation loss: 0.1032
Epoch 17



Validation accuracy: 0.9673, validation loss: 0.1030
Epoch 18



Validation accuracy: 0.9697, validation loss: 0.0925
Epoch 19



Validation accuracy: 0.9715, validation loss: 0.0894
Epoch 20



Validation accuracy: 0.9727, validation loss: 0.0832


In [36]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9762, device='cuda:0'), tensor(0.0784, device='cuda:0'))



Valid: (tensor(0.9727, device='cuda:0'), tensor(0.0832, device='cuda:0'))


In [37]:
optimizer.accelerate()

In [38]:
optimizer.store_parameters()
model.cuda()

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (6): ReLU()
  (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (8): ReLU()
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=1024, out_features=128, bias=True)
  (12): ReLU()
  (13): Linear(in_features=128, out_features=10, bias=True)
  (14): LogSoftmax(dim=-1)
)

In [39]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9753, device='cuda:0'), tensor(0.0809, device='cuda:0'))



Valid: (tensor(0.9725, device='cuda:0'), tensor(0.0887, device='cuda:0'))


## Only linear

In [9]:
model = make_model()
model.cuda()
conv_group = {
    "params": [param for child in list(model.children())[:10] for param in child.parameters()],
    "method": None
}
fc_group = {
    "params": [param for child in list(model.children())[10:] for param in child.parameters()]
}
groups = [conv_group, fc_group]
optimizer = AcceleratedSGD(groups, 1e-3, k=10, momentum=0.5, weight_decay=1e-5, mode="epoch")

In [11]:
epochs = 20

for epoch in range(epochs):
    print("Epoch", epoch+1)
    train_epoch()
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

Epoch 1


/opt/miniconda3/envs/torch/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/miniconda3/envs/torch/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



Validation accuracy: 0.1137, validation loss: 2.3022
Epoch 2



Validation accuracy: 0.1137, validation loss: 2.3002
Epoch 3



Validation accuracy: 0.1137, validation loss: 2.2977
Epoch 4


...


Validation accuracy: 0.9588, validation loss: 0.1290
Epoch 17



Validation accuracy: 0.9535, validation loss: 0.1435
Epoch 18



Validation accuracy: 0.9614, validation loss: 0.1194
Epoch 19



Validation accuracy: 0.9585, validation loss: 0.1291
Epoch 20



Validation accuracy: 0.9673, validation loss: 0.1044


In [12]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9728, device='cuda:0'), tensor(0.0890, device='cuda:0'))



Valid: (tensor(0.9673, device='cuda:0'), tensor(0.1044, device='cuda:0'))


In [13]:
optimizer.accelerate()

In [14]:
optimizer.store_parameters()
model.cuda()

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (6): ReLU()
  (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (8): ReLU()
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=1024, out_features=128, bias=True)
  (12): ReLU()
  (13): Linear(in_features=128, out_features=10, bias=True)
  (14): LogSoftmax(dim=-1)
)

In [15]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9714, device='cuda:0'), tensor(0.0927, device='cuda:0'))



Valid: (tensor(0.9655, device='cuda:0'), tensor(0.1098, device='cuda:0'))


## Linear + conv separately

In [16]:
model = make_model()
model.cuda()
conv_group = {
    "params": [param for child in list(model.children())[:10] for param in child.parameters()],
}
fc_group = {
    "params": [param for child in list(model.children())[10:] for param in child.parameters()]
}
groups = [conv_group, fc_group]
optimizer = AcceleratedSGD(groups, 1e-3, k=10, momentum=0.5, weight_decay=1e-5, mode="epoch")

In [17]:
epochs = 20

for epoch in range(epochs):
    print("Epoch", epoch+1)
    train_epoch()
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

Epoch 1



Validation accuracy: 0.1006, validation loss: 2.2953
Epoch 2



Validation accuracy: 0.3298, validation loss: 2.2731
Epoch 3



Validation accuracy: 0.6801, validation loss: 1.5324
Epoch 4



Validation accuracy: 0.8598, validation loss: 0.4727
Epoch 5


...


Validation accuracy: 0.9664, validation loss: 0.1036
Epoch 17



Validation accuracy: 0.9672, validation loss: 0.1045
Epoch 18



Validation accuracy: 0.9693, validation loss: 0.0931
Epoch 19



Validation accuracy: 0.9723, validation loss: 0.0904
Epoch 20



Validation accuracy: 0.9673, validation loss: 0.1015


In [18]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9740, device='cuda:0'), tensor(0.0840, device='cuda:0'))



Valid: (tensor(0.9673, device='cuda:0'), tensor(0.1015, device='cuda:0'))


In [19]:
optimizer.accelerate()

In [20]:
optimizer.store_parameters()
model.cuda()

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (6): ReLU()
  (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (8): ReLU()
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=1024, out_features=128, bias=True)
  (12): ReLU()
  (13): Linear(in_features=128, out_features=10, bias=True)
  (14): LogSoftmax(dim=-1)
)

In [21]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9708, device='cuda:0'), tensor(0.0919, device='cuda:0'))



Valid: (tensor(0.9682, device='cuda:0'), tensor(0.1057, device='cuda:0'))
